In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/all.csv')
df=df[["ESR1.log2.tpm" ,"PGR.log2.tpm" , "IL12RB2","HORMAD1","LEMD1","AGR3","GFRA1","ANTXRL",'NXNL2',"SPATA4",'RNF182',
      "TP53"  , "COL5A1", "GATA3",'LYST',
      "Danaher.Mast.cells" , "Act.CD4"  , "MDSC",'NK cells','Tem.CD8','CD8 T cells',"Danaher.Neutrophils","Danaher.NK.cells","Monocytic lineage" ,'Danaher.Exhausted.CD8','Danaher.CD8.T.cells','Cytotoxic lymphocytes','Danaher.Cytotoxic.cells',
      "Taxane.FinalScore","Swanton.PaclitaxelScore", "Taxane.MitosisScore" ,"Swanton.MitosisScore" ,
                                     'IDO1','TIDE.TAM.M2',"ESC.ssgsea.notnorm","ESC.ssgsea.norm"        
,"GGI.ssgsea.notnorm","resp.pCR"]]

In [ ]:
X=df.iloc[:,0:-1]
y=df.iloc[:,-1]

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
class SelectAtMostKBest(SelectKBest):
    def _check_params(self, X, y):
        if not (self.k == "all" or 0 <= self.k <= X.shape[1]):
            # set k to "all" (skip feature selection), if less than k features are available
            self.k = "all"

In [ ]:
AUC_svc=[]
for feature in range(X.shape[1]+1):
    # 用特定特征删除后的特征集
    #X_drop = X.drop(X.columns[feature], axis=1)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    if feature < X.shape[1]:
        X_drop = X.drop(X.columns[feature], axis=1)
    else:
        X_drop = X
        
    scaler = StandardScaler()
    kbest = SelectAtMostKBest(score_func=f_classif)
    svc = SVC(random_state=1, max_iter=-1, probability=True)
    pipe = Pipeline(steps=[('scaler', scaler), ('kbest', kbest), ('svc', svc)])
    # Parameter ranges
    param_grid = { 'kbest__k': np.arange(2,X.shape[1],1),
                    'svc__kernel': ['rbf','sigmoid','linear'],
                    'svc__gamma': np.logspace(-9,-2,60),
                    'svc__C': np.logspace(-3,3,60)}
    # Optimisation
    search = RandomizedSearchCV(pipe, param_grid, cv=skf, scoring='roc_auc', return_train_score=True, n_jobs=10, verbose=0, n_iter=1000, random_state=0)

    search.fit(X_drop,y)
    
    best_params = search.best_params_
    best_score = search.best_score_
    AUC_svc.append(best_score)
    
    print(best_params,best_score)

{'svc__kernel': 'rbf', 'svc__gamma': 4.581597669054491e-08, 'svc__C': 4.581597669054491, 'kbest__k': 30} 0.875
{'svc__kernel': 'sigmoid', 'svc__gamma': 2.0187602546790387e-08, 'svc__C': 626.0516572014828, 'kbest__k': 35} 0.875
{'svc__kernel': 'rbf', 'svc__gamma': 7.912342618981326e-08, 'svc__C': 0.04237587160604064, 'kbest__k': 36} 0.8767316017316016
{'svc__kernel': 'sigmoid', 'svc__gamma': 8.895134973108219e-09, 'svc__C': 9.249147277217336, 'kbest__k': 33} 0.8732683982683982
{'svc__kernel': 'sigmoid', 'svc__gamma': 3.1011689265747754e-07, 'svc__C': 47.639380104013405, 'kbest__k': 30} 0.8686688311688311
{'svc__kernel': 'sigmoid', 'svc__gamma': 3.1011689265747754e-07, 'svc__C': 60.20894493336138, 'kbest__k': 34} 0.873917748917749
{'svc__kernel': 'rbf', 'svc__gamma': 7.912342618981326e-08, 'svc__C': 0.04237587160604064, 'kbest__k': 36} 0.8687229437229437
{'svc__kernel': 'rbf', 'svc__gamma': 2.982471286216894e-09, 'svc__C': 153.61749466718297, 'kbest__k': 34} 0.8665584415584415
{'svc__ker

In [ ]:
len(AUC_svc)

38

In [ ]:
AUC_svc

[0.875,
 0.875,
 0.8767316017316016,
 0.8732683982683982,
 0.8686688311688311,
 0.873917748917749,
 0.8687229437229437,
 0.8665584415584415,
 0.8772186147186147,
 0.8706709956709956,
 0.8665043290043289,
 0.8811147186147187,
 0.8783008658008657,
 0.8761904761904763,
 0.875,
 0.8818722943722943,
 0.878896103896104,
 0.8751623376623376,
 0.8762987012987015,
 0.8713203463203463,
 0.8801948051948051,
 0.8675865800865801,
 0.8766233766233766,
 0.8739718614718616,
 0.8753787878787879,
 0.8756493506493506,
 0.8766774891774892,
 0.8774891774891774,
 0.8778138528138528,
 0.8778138528138528,
 0.8742965367965368,
 0.8742965367965368,
 0.8743506493506494,
 0.8749458874458874,
 0.8813311688311689,
 0.8813311688311689,
 0.8766774891774892,
 0.8761904761904763]